<a href="https://colab.research.google.com/github/HernanHz/Clase-1/blob/main/Sistema_de_Biblioteca_Fase_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import deque
from queue import Queue

# Clase Libro
class Libro:
    def __init__(self, titulo, autor, categoria):
        self.titulo = titulo
        self.autor = autor
        self.categoria = categoria
        self.disponible = True
        self.dias_prestamo = 0

    def get_titulo(self):
        return self.titulo

    def is_disponible(self):
        return self.disponible

    def prestar(self, dias):
        self.disponible = False
        self.dias_prestamo = dias

    def devolver(self):
        self.disponible = True
        self.dias_prestamo = 0

    def __str__(self):
        estado = "Sí" if self.disponible else "No"
        return f"Título: {self.titulo}, Autor: {self.autor}, Categoría: {self.categoria}, Disponible: {estado}"

# Nodo del árbol de libros
class NodoLibro:
    def __init__(self, libro):
        self.libro = libro
        self.izquierdo = None
        self.derecho = None

# Árbol de libros
class ArbolLibros:
    def __init__(self):
        self.raiz = None

    def insertar(self, libro):
        self.raiz = self._insertar_rec(self.raiz, libro)

    def _insertar_rec(self, nodo, libro):
        if nodo is None:
            return NodoLibro(libro)
        if libro.titulo.lower() < nodo.libro.titulo.lower():
            nodo.izquierdo = self._insertar_rec(nodo.izquierdo, libro)
        elif libro.titulo.lower() > nodo.libro.titulo.lower():
            nodo.derecho = self._insertar_rec(nodo.derecho, libro)
        return nodo

    def buscar(self, titulo):
        return self._buscar_rec(self.raiz, titulo.lower())

    def _buscar_rec(self, nodo, titulo):
        if nodo is None:
            return None
        if titulo == nodo.libro.titulo.lower():
            return nodo.libro
        elif titulo < nodo.libro.titulo.lower():
            return self._buscar_rec(nodo.izquierdo, titulo)
        else:
            return self._buscar_rec(nodo.derecho, titulo)

    def inorden(self):
        self._inorden_rec(self.raiz)

    def _inorden_rec(self, nodo):
        if nodo:
            self._inorden_rec(nodo.izquierdo)
            print(nodo.libro)
            self._inorden_rec(nodo.derecho)

# Usuario y árbol de usuarios
class Usuario:
    def __init__(self, nombre):
        self.nombre = nombre

    def get_nombre(self):
        return self.nombre

    def __str__(self):
        return f"Nombre: {self.nombre}"

class NodoUsuario:
    def __init__(self, usuario):
        self.usuario = usuario
        self.izquierdo = None
        self.derecho = None

class ArbolUsuarios:
    def __init__(self):
        self.raiz = None

    def insertar(self, usuario):
        self.raiz = self._insertar_rec(self.raiz, usuario)

    def _insertar_rec(self, nodo, usuario):
        if nodo is None:
            return NodoUsuario(usuario)
        if usuario.nombre.lower() < nodo.usuario.nombre.lower():
            nodo.izquierdo = self._insertar_rec(nodo.izquierdo, usuario)
        elif usuario.nombre.lower() > nodo.usuario.nombre.lower():
            nodo.derecho = self._insertar_rec(nodo.derecho, usuario)
        return nodo

    def buscar(self, nombre):
        return self._buscar_rec(self.raiz, nombre.lower())

    def _buscar_rec(self, nodo, nombre):
        if nodo is None:
            return None
        if nombre == nodo.usuario.nombre.lower():
            return nodo.usuario
        elif nombre < nodo.usuario.nombre.lower():
            return self._buscar_rec(nodo.izquierdo, nombre)
        else:
            return self._buscar_rec(nodo.derecho, nombre)

    def inorden(self):
        self._inorden_rec(self.raiz)

    def _inorden_rec(self, nodo):
        if nodo:
            self._inorden_rec(nodo.izquierdo)
            print(nodo.usuario)
            self._inorden_rec(nodo.derecho)

# Árbol de categorías
class ArbolCategorias:
    def __init__(self):
        self.categorias = {}  # Diccionario: clave = categoría, valor = lista de libros

    def agregar_libro(self, libro):
        categoria = libro.categoria.lower()
        if categoria not in self.categorias:
            self.categorias[categoria] = []
        self.categorias[categoria].append(libro)

    def mostrar_categorias(self):
        print("Categorías registradas:")
        for categoria in sorted(self.categorias.keys()):
            print(f"- {categoria.capitalize()} ({len(self.categorias[categoria])} libros)")

    def mostrar_libros_por_categoria(self, categoria):
        categoria = categoria.lower()
        if categoria in self.categorias:
            print(f"\nLibros en la categoría '{categoria.capitalize()}':")
            for libro in self.categorias[categoria]:
                print(libro)
        else:
            print("Categoría no encontrada.")

# Clase principal Biblioteca
class Biblioteca:
    def __init__(self):
        self.arbol_libros = ArbolLibros()
        self.arbol_usuarios = ArbolUsuarios()
        self.arbol_categorias = ArbolCategorias()
        self.lista_espera = Queue()
        self.historial_prestamos = []

    def agregar_libro(self, titulo, autor, categoria):
        libro = Libro(titulo, autor, categoria)
        self.arbol_libros.insertar(libro)
        self.arbol_categorias.agregar_libro(libro)
        print("Libro agregado correctamente al sistema.")

    def registrar_usuario(self, nombre):
        if self.arbol_usuarios.buscar(nombre):
            print("El usuario ya está registrado.")
        else:
            self.arbol_usuarios.insertar(Usuario(nombre))
            print("Usuario registrado con éxito.")

    def prestar_libro(self, titulo, nombre_usuario, dias):
        libro = self.arbol_libros.buscar(titulo)
        if libro and libro.is_disponible():
            libro.prestar(dias)
            self.historial_prestamos.append(f"Libro '{titulo}' prestado a {nombre_usuario} por {dias} días.")
            print("Libro prestado con éxito.")
        else:
            print("Libro no disponible. Usuario agregado a la lista de espera.")
            self.lista_espera.put(Usuario(nombre_usuario))

    def devolver_libro(self, titulo):
        libro = self.arbol_libros.buscar(titulo)
        if libro and not libro.is_disponible():
            libro.devolver()
            self.historial_prestamos.append(f"Libro '{titulo}' devuelto.")
            print("Libro devuelto con éxito.")
            if not self.lista_espera.empty():
                siguiente_usuario = self.lista_espera.get()
                print(f"Libro ahora disponible para {siguiente_usuario.get_nombre()}")
        else:
            print("Libro no encontrado o ya está disponible.")

    def mostrar_historial_prestamos(self):
        print("Historial de Préstamos:")
        while self.historial_prestamos:
            print(self.historial_prestamos.pop())

    def mostrar_libros(self):
        print("Listado de libros en orden alfabético:")
        self.arbol_libros.inorden()

    def mostrar_usuarios(self):
        print("Listado de usuarios en orden alfabético:")
        self.arbol_usuarios.inorden()

    def mostrar_categorias(self):
        self.arbol_categorias.mostrar_categorias()

    def mostrar_libros_por_categoria(self, categoria):
        self.arbol_categorias.mostrar_libros_por_categoria(categoria)

# Menú interactivo
def main():
    biblioteca = Biblioteca()

    while True:
        print("\n--- Menú de Biblioteca ---")
        print("1. Agregar libro")
        print("2. Registrar usuario")
        print("3. Prestar libro")
        print("4. Devolver libro")
        print("5. Ver historial de préstamos")
        print("6. Ver libros ordenados")
        print("7. Ver usuarios ordenados")
        print("8. Ver categorías registradas")
        print("9. Ver libros por categoría")
        print("10. Salir")
        opcion = input("Seleccione una opción: ")

        if opcion == '1':
            titulo = input("Título: ")
            autor = input("Autor: ")
            categoria = input("Categoría: ")
            biblioteca.agregar_libro(titulo, autor, categoria)

        elif opcion == '2':
            nombre = input("Nombre del usuario: ")
            biblioteca.registrar_usuario(nombre)

        elif opcion == '3':
            titulo = input("Título del libro a prestar: ")
            nombre_usuario = input("Nombre del usuario: ")
            dias = int(input("Días de préstamo: "))
            biblioteca.prestar_libro(titulo, nombre_usuario, dias)

        elif opcion == '4':
            titulo = input("Título del libro a devolver: ")
            biblioteca.devolver_libro(titulo)

        elif opcion == '5':
            biblioteca.mostrar_historial_prestamos()

        elif opcion == '6':
            biblioteca.mostrar_libros()

        elif opcion == '7':
            biblioteca.mostrar_usuarios()

        elif opcion == '8':
            biblioteca.mostrar_categorias()

        elif opcion == '9':
            categoria = input("Nombre de la categoría: ")
            biblioteca.mostrar_libros_por_categoria(categoria)

        elif opcion == '10':
            print("Saliendo del sistema. ¡Hasta pronto!")
            break

        else:
            print("Opción no válida, intente de nuevo.")

# Ejecutar el programa
if __name__ == "__main__":
    main()



--- Menú de Biblioteca ---
1. Agregar libro
2. Registrar usuario
3. Prestar libro
4. Devolver libro
5. Ver historial de préstamos
6. Ver libros ordenados
7. Ver usuarios ordenados
8. Ver categorías registradas
9. Ver libros por categoría
10. Salir
Seleccione una opción: 1
Título: El principito
Autor: Antoine de Saint-Exupéry
Categoría: Novela
Libro agregado correctamente al sistema.

--- Menú de Biblioteca ---
1. Agregar libro
2. Registrar usuario
3. Prestar libro
4. Devolver libro
5. Ver historial de préstamos
6. Ver libros ordenados
7. Ver usuarios ordenados
8. Ver categorías registradas
9. Ver libros por categoría
10. Salir
Seleccione una opción: 1
Título: Cien años de soledad
Autor: Gabriel García Márquez
Categoría: Novela
Libro agregado correctamente al sistema.

--- Menú de Biblioteca ---
1. Agregar libro
2. Registrar usuario
3. Prestar libro
4. Devolver libro
5. Ver historial de préstamos
6. Ver libros ordenados
7. Ver usuarios ordenados
8. Ver categorías registradas
9. Ver lib